In [91]:
import requests
from bs4 import BeautifulSoup as soup
import os.path
import os
import json
from datetime import datetime
from datetime import timedelta


# https://api.slack.com/apps/APSE1SZPZ/incoming-webhooks?
class SlackNotifier():
    def __init__(self, url, interval_in_s):
        self.slack_hook_url = url
        self.tdelta = timedelta(seconds=interval_in_s)
        self.lasttime = {}
    def call(self, Id, message, forced=False):
        if forced:
            requests.post(self.slack_hook_url, json={"text":message})
            self.lasttime[Id] = datetime.now()
        elif not self.is_just_called(Id):
            requests.post(self.slack_hook_url, json={"text":message})
            self.lasttime[Id] = datetime.now()
    def is_just_called(self, Id):
        if not self.lasttime:
            return False
        elif Id in self.lasttime.keys():
            return datetime.now() < self.lasttime[Id] + self.tdelta        
        else:
            return False

def extract_today_created_time(item_time):
    if "Heute" in item_time:
        index = item_time.find("Heute,")
        created_hours = int(item_time[index+7:index + 9])
        created_minutes = int(item_time[index+10:index + 12])
        return True, timedelta(hours=created_hours, minutes=created_minutes)
    else:
        return False, item_time.replace("\n","").replace(" ","")
    
def is_new(created_time, interval_in_s=30000):
    """ Post can be updated later in 3 minutes => use default interval is 4' = 240"
    """
    tdelta = timedelta(seconds=int(interval_in_s))
    now_hours = datetime.now().hour
    now_minutes = datetime.now().minute
    now_seconds = datetime.now().second
    now = timedelta(hours=now_hours, minutes=now_minutes, seconds=now_seconds)
    return now < created_time + tdelta

In [92]:
notifier = SlackNotifier(os.environ['SLACK_HOOK_URL'], 10)

In [93]:
notifier.call(0, "FOR EBAY-KLEINANZEIGEN: NEW MESSAGE.")

In [212]:
# Get information from KVR

termin_url = "https://www46.muenchen.de/termin/index.php?cts=1080627"

headers1 = {
    'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1',
    'Connection' : 'keep-alive',
    'Cache-Control' : 'max-age=0',
    'Upgrade-Insecure-Requests' : '1',
    'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding' : 'gzip, deflate, br',
    'Accept-Language' : 'en-US,en;q=0.9,de;q=0.8,fr;q=0.7,vi;q=0.6',
    'Cookie' : '_et_coid=e32c2d08fe2aec5f79ee9875c9f20463'
}

r1 = requests.get(termin_url, headers=headers1)
page = r1.content
doc = soup(page, "html.parser")
element = doc.findAll('input', {"name": "__ncforminfo"})[0]

data = {}
data['step'] = 'WEB_APPOINT_SEARCH_BY_CASETYPES'
data['CASETYPES[Aufenthaltserlaubnis Blaue Karte EU]'] = '1'
data['__ncforminfo'] = element['value']

headers2 = {
    'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1',
    'Connection' : 'keep-alive',
    'Cache-Control' : 'max-age=0',
    'Upgrade-Insecure-Requests' : '1',
    'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding' : 'gzip, deflate, br',
    'Accept-Language' : 'en-US,en;q=0.9,de;q=0.8,fr;q=0.7,vi;q=0.6',
}

r2 = requests.post(termin_url, cookies=r1.cookies, headers=headers2, data=data)
page = r2.content
doc = soup(page, "html.parser")
doc

<!DOCTYPE html>

<html lang="de">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="./img/favicon.gif" rel="icon" type="image/gif"/>
<meta content="width=device-width, initial-scale=1.0, max-scale=1.5, user-scalable=no" name="viewport"/>
<link href="./tpl/Muc/CSS/Default.tpl.css" rel="STYLESHEET" type="text/css"/>
<link href="./tpl/Muc/CSS/responsive.css" rel="STYLESHEET" type="text/css"/>
<link href="./tpl/Muc/CSS/reader.css" media="reader" rel="STYLESHEET" type="text/css"/>
<!-- CUSTOM_CSS -->
<script src="./js/jquery-1.11.2.min.js" type="text/javascript"></script>
<script src="./js/barr.js" type="text/javascript"></script>
<title>Terminvereinbarung - Auswahl der Dienstleistung</title>
<script src="./js/appointPicker.js" type="text/javascript"></script>
<!--[if lt IE 8]>
          <style type="text/css">@import url(./tpl/Var_2M_loc/CSS/ie8.css);</style>
        <![endif]-->
<script>
            var jsonAppoints = '{"Termin Wartezone SCIF":{"capti

In [6]:
lat = "48.1151649"
lng = "11.6981558"
queries = ["ikea skarsta", "bose revolve"]
queries = ["+".join(query.split(" ")) for query in queries]
distances = [20, 1] # in km
page_num = 0
interval_in_s = 30

assert len(distances) == len(queries), "Number of queries and distance is different."

In [7]:
i = 0
query = queries[i]
dist = distances[i]

In [8]:
url_for_location_id = "https://m.ebay-kleinanzeigen.de/s-ort-vorschlag.json?lat="+ lat + "&lng=" + lng
result = requests.get(url_for_location_id)
page = result.text
locationId = json.loads(page)[0]['id']

In [9]:
url = "https://m.ebay-kleinanzeigen.de/s-suche-veraendern?locationId=" + str(locationId) + \
        "&distance=" + str(dist) + \
        "&categoryId=&minPrice=&maxPrice=&adType=&posterType="+ \
        "&q=" + query

result = requests.get(url)
page = result.text
doc = soup(page, "html.parser")
links = [element.get('content') for element in doc.find_all('meta')]
link = links[6]

link0, link1 = link.split("distance")
url = link0 + "page=" + str(page_num) + "&distance" + link1

In [10]:
result = requests.get(url)
page = result.text
doc = soup(page, "html.parser")
items = [element for element in doc.find_all('div', {"class": "adlist--item--descarea"})]

In [8]:
# items

In [9]:
if len(items) > 0:
    item = items[0] # get last item
    item_time = item.find('div', {"class": "adlist--item--info--date"}).contents[0]
    is_today, created_time = extract_today_created_time(item_time)
    if is_today:
        print("Today {}".format(created_time))
        if is_new(created_time):
            notifier.call("FOR EBAY-KLEINANZEIGEN: NEW MESSAGE.", forced=False)
            print("SEND")
    else:
        print(created_time)
            

Today 17:32:00
SEND


# Delete process script

In [54]:
import os

if os.path.exists("/tmp/ebay-kleinanzeigen.pid"):
    with open("/tmp/ebay-kleinanzeigen.pid", "r") as f:
        current_pid = f.readline()
        current_pid = current_pid.split("\n")[0]
        kill_syntax = "kill -9 {}".format(current_pid)
        os.system(kill_syntax)
        if os.path.exists("/tmp/ebay-kleinanzeigen.pid.lock"):
            os.remove("/tmp/ebay-kleinanzeigen.pid.lock")